# 05 - Pre-processing After Removing Features with RFE

In [20]:
import math
import pandas as pd
import numpy as np
import scipy.stats as sp
from joblib import dump, load

from utils import dataframe_utils,preprocess_utils

In [23]:
df_covid = pd.read_excel('datasets/dataset.xlsx')
df_covid

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.022340,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5639,ae66feb9e4dc3a0,3,positive,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5640,517c2834024f3ea,17,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5641,5c57d6037fe266d,4,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5642,c20c44766f28291,10,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
list(df_covid.columns)

['Patient ID',
 'Patient age quantile',
 'SARS-Cov-2 exam result',
 'Patient addmited to regular ward (1=yes, 0=no)',
 'Patient addmited to semi-intensive unit (1=yes, 0=no)',
 'Patient addmited to intensive care unit (1=yes, 0=no)',
 'Hematocrit',
 'Hemoglobin',
 'Platelets',
 'Mean platelet volume ',
 'Red blood Cells',
 'Lymphocytes',
 'Mean corpuscular hemoglobin concentration\xa0(MCHC)',
 'Leukocytes',
 'Basophils',
 'Mean corpuscular hemoglobin (MCH)',
 'Eosinophils',
 'Mean corpuscular volume (MCV)',
 'Monocytes',
 'Red blood cell distribution width (RDW)',
 'Serum Glucose',
 'Respiratory Syncytial Virus',
 'Influenza A',
 'Influenza B',
 'Parainfluenza 1',
 'CoronavirusNL63',
 'Rhinovirus/Enterovirus',
 'Mycoplasma pneumoniae',
 'Coronavirus HKU1',
 'Parainfluenza 3',
 'Chlamydophila pneumoniae',
 'Adenovirus',
 'Parainfluenza 4',
 'Coronavirus229E',
 'CoronavirusOC43',
 'Inf A H1N1 2009',
 'Bordetella pertussis',
 'Metapneumovirus',
 'Parainfluenza 2',
 'Neutrophils',
 'Urea',

In [28]:
df_covid.shape

(5644, 19)

In [27]:
df_covid = df_covid[[
    'Patient age quantile', 
    'SARS-Cov-2 exam result', 
    'Platelets',
    'Mean platelet volume ', 
    'Red blood Cells', 
    'Lymphocytes',
    'Mean corpuscular hemoglobin concentration\xa0(MCHC)', 
    'Leukocytes',
    'Basophils', 
    'Mean corpuscular hemoglobin (MCH)', 
    'Eosinophils',
    'Mean corpuscular volume (MCV)', 
    'Monocytes',
    'Red blood cell distribution width (RDW)', 
    'Influenza B',
    'CoronavirusNL63', 
    'Rhinovirus/Enterovirus', 
    'Coronavirus HKU1',
    'Inf A H1N1 2009']]

### Checking for missing values

In [41]:
df_counts = df_covid.count()
display(df_counts)

Patient age quantile                                5644
SARS-Cov-2 exam result                              5644
Platelets                                            602
Mean platelet volume                                 599
Red blood Cells                                      602
Lymphocytes                                          602
Mean corpuscular hemoglobin concentration (MCHC)     602
Leukocytes                                           602
Basophils                                            602
Mean corpuscular hemoglobin (MCH)                    602
Eosinophils                                          602
Mean corpuscular volume (MCV)                        602
Monocytes                                            601
Red blood cell distribution width (RDW)              602
Influenza B                                         1354
CoronavirusNL63                                     1352
Rhinovirus/Enterovirus                              1352
Coronavirus HKU1               

In [44]:
df_covid_subset_final = df_covid.dropna()
df_covid_subset_final.shape

(362, 19)

Even when taking the most relevant features from the original dataset, the result is pretty much the same size as the previous dataset.

In [42]:
def drop_by_data_threshold(df,threshold,axis):
    print(f'threshold (%) = {threshold * 100}')
    if axis == 'row': axis = 1
    elif axis == 'col' or axis == 'column': axis = 0

    max_values = df.shape[axis]
    value_threshold = int(threshold * max_values)
    mask_values_to_keep = df.count(axis = axis) >= value_threshold
    print('threshold (data points) =',value_threshold)
    if axis == 1: return df[mask_values_to_keep]
    elif axis == 0: return df.loc[:,mask_values_to_keep.values]